<a href="https://colab.research.google.com/github/kym0323/National_Petition_analysis/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D%EC%B2%AD%EC%9B%90%EA%B2%8C%EC%8B%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 48.6MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기



rawdata_downloaded = drive.CreateFile({'id': '1IQEc2ibk9fWat0uszpiEWuJlmlhY1Ydv'})
rawdata_downloaded.GetContentFile('rawdata.xlsx')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_excel('rawdata.xlsx')

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,number,progress,title,like,contents,category,sday,eday,proponent
0,28,청원종료,소액주주를 보호해주십시오,2137,** 존경하옵는 문재인대통령님께 \n\n\n저는 중국원양자원이라는 KOSPI상장사...,경제민주화,2017-08-19,2017-08-26,naver
1,34,청원종료,『국가유공자 등 예우 및 지원에 관한법률』상「6.25전몰군경 자녀수당」의 불합리한 ...,679,(현황)\n우리들 아버지께서는 67여년전 북의 남침으로 조국이 위기에 처했을 때 젊...,기타,2017-08-19,2017-09-18,naver
2,43,청원종료,기간제 교사의 정규직화를 반대합니다.,11293,"대통령님, 안녕하세요. 저는 임용을 준비하고 수험생입니다. 처음 기간제 정규직화 된...",육아/교육,2017-08-19,2017-09-18,naver
3,46,청원종료,기간제교사의 정규직화를 반대합니다.,1933,"대통령님, 안녕하세요. 저는 임용을 준비하고 수험생입니다. 처음 기간제 정규직화 된...",육아/교육,2017-08-19,2017-09-18,naver
4,50,청원종료,치과위생사 국가고시 날짜 변경 억울합니다,1251,안녕하십니까? 대통령님 간단하게 제소개 부터 하겠습니다. 저는 치위생과 3학년 학생...,일자리,2017-08-19,2017-10-18,naver
...,...,...,...,...,...,...,...,...,...
11892,589992,청원종료,주식 양도세 확대는 부당합니다.,87324,안녕하세요. 주식을 재테크 수단으로 하고 있는 대한민국 국민입니다. 대통령님 최근 ...,행정,2020-06-23,2020-07-23,kakao
11893,589995,청원종료,서민을 망하게하는 617부동산대책,2751,안녕하세요. 저는 인천 부평에 살고있는 40대 주부입니다.\n저희는 얼마전 부평**...,교통/건축/국토,2020-06-23,2020-07-23,naver
11894,589997,청원종료,617 부동산 대책으로 대출규제를 소급적용하는게 과연 상식에 맞다고 생각하십니까?,3701,저는 얼마전까지 비규제지역이었던 인천에 분양권을 2개 소유한 사람입니다. \n현 정...,교통/건축/국토,2020-06-23,2020-07-23,naver
11895,589999,청원종료,2주택자 중도금대출불가 소급적용은 위헌적 행정조치입니다,5002,저는 2주택자입니다. \n20.5월경 인천 중구 영종도는 비규제지역이라 아파트 분양...,행정,2020-06-23,2020-07-23,naver


In [ ]:
원본데이터 = 원본데이터.dropna(axis=0)

In [ ]:
원본데이터

,number,progress,title,like,contents,category,sday,eday,proponent
0,28,청원종료,소액주주를 보호해주십시오,2137,** 존경하옵는 문재인대통령님께 \n\n\n저는 중국원양자원이라는 KOSPI상장사...,경제민주화,2017-08-19,2017-08-26,naver
1,34,청원종료,『국가유공자 등 예우 및 지원에 관한법률』상「6.25전몰군경 자녀수당」의 불합리한 ...,679,(현황)\n우리들 아버지께서는 67여년전 북의 남침으로 조국이 위기에 처했을 때 젊...,기타,2017-08-19,2017-09-18,naver
2,43,청원종료,기간제 교사의 정규직화를 반대합니다.,11293,"대통령님, 안녕하세요. 저는 임용을 준비하고 수험생입니다. 처음 기간제 정규직화 된...",육아/교육,2017-08-19,2017-09-18,naver
3,46,청원종료,기간제교사의 정규직화를 반대합니다.,1933,"대통령님, 안녕하세요. 저는 임용을 준비하고 수험생입니다. 처음 기간제 정규직화 된...",육아/교육,2017-08-19,2017-09-18,naver
4,50,청원종료,치과위생사 국가고시 날짜 변경 억울합니다,1251,안녕하십니까? 대통령님 간단하게 제소개 부터 하겠습니다. 저는 치위생과 3학년 학생...,일자리,2017-08-19,2017-10-18,naver
...,...,...,...,...,...,...,...,...,...
11892,589992,청원종료,주식 양도세 확대는 부당합니다.,87324,안녕하세요. 주식을 재테크 수단으로 하고 있는 대한민국 국민입니다. 대통령님 최근 ...,행정,2020-06-23,2020-07-23,kakao
11893,589995,청원종료,서민을 망하게하는 617부동산대책,2751,안녕하세요. 저는 인천 부평에 살고있는 40대 주부입니다.\n저희는 얼마전 부평**...,교통/건축/국토,2020-06-23,2020-07-23,naver
11894,589997,청원종료,617 부동산 대책으로 대출규제를 소급적용하는게 과연 상식에 맞다고 생각하십니까?,3701,저는 얼마전까지 비규제지역이었던 인천에 분양권을 2개 소유한 사람입니다. \n현 정...,교통/건축/국토,2020-06-23,2020-07-23,naver
11895,589999,청원종료,2주택자 중도금대출불가 소급적용은 위헌적 행정조치입니다,5002,저는 2주택자입니다. \n20.5월경 인천 중구 영종도는 비규제지역이라 아파트 분양...,행정,2020-06-23,2020-07-23,naver


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[4] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[5] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,**,Punctuation,경제민주화,1
1,존경,Noun,경제민주화,1
2,하옵는,Verb,경제민주화,1
3,문재인,Noun,경제민주화,1
4,대통령,Noun,경제민주화,1
...,...,...,...,...
1145,\n\n,Foreign,일자리,1
1146,진심,Noun,일자리,1
1147,으로,Josa,일자리,1
1148,부탁드립니다,Adjective,일자리,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,**,Punctuation,경제민주화,1
1,존경,Noun,경제민주화,1
2,하옵는,Verb,경제민주화,1
3,문재인,Noun,경제민주화,1
4,대통령,Noun,경제민주화,1
...,...,...,...,...
1145,\n\n,Foreign,일자리,1
1146,진심,Noun,일자리,1
1147,으로,Josa,일자리,1
1148,부탁드립니다,Adjective,일자리,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소   품사       분류         
\t    Foreign  경제민주화          61
               교통/건축/국토       33
               기타              4
               문화/예술/체육/언론     4
               미래              8
                              ..
🤬😡    Foreign  정치개혁            1
🤬😡👿😠  Foreign  정치개혁            1
🤬🤬🤬   Foreign  외교/통일/국방        1
󰋎     Foreign  교통/건축/국토        1
󰋏     Foreign  교통/건축/국토        1
Name: 카운트, Length: 420877, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소  품사      분류              
\t   Foreign 경제민주화         61
             교통/건축/국토      33
             기타             4
             문화/예술/체육/언론    4
             미래             8
...                       ...
🤬😡   Foreign 정치개혁           1
🤬😡👿😠 Foreign 정치개혁           1
🤬🤬🤬  Foreign 외교/통일/국방       1
󰋎    Foreign 교통/건축/국토       1
󰋏    Foreign 교통/건축/국토       1

[420877 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과_si.csv', header='true', encoding='utf-8-sig')